In [29]:
import geopandas as gpd
import pandas as pd
import re
import plotly.express as px

from shapely.wkt import loads
from glob import glob
from plotly.offline import plot
from shapely.geometry import Point


In [164]:
# 행정동 정보 
files = glob("*.shp")

shp = pd.concat([gpd.read_file(file,encoding = 'cp949') for file in files]).reset_index(drop = True)

df = pd.read_csv("당신의 사는곳을 알려주세요 9572172efb494f81a1243b25bad749d8.csv")
df = df[df.columns[:3]].dropna()

In [168]:
df[df.시도명 == '양주시'] #.. 으으 ㅂㄷㅂㄷ..

,Name,시도명,시군구명
3,XXX,양주시,백석읍


In [171]:
df.loc[df.시도명 == '양주시','시도명'] = '경기도'
df.loc[df.시군구명 == '백석읍','시군구명'] = '양주시'

# 띄어쓰기 제거 

In [176]:
megas = df.시도명.apply(lambda x: x.replace(" ",""))

sggs= df.시군구명.apply(lambda x: x.replace(" ",''))

# 중심점 계산 후 리스트로 저장 

In [180]:
centroid = [shp[((shp.MEGA_NM.apply(lambda x: re.search(mega,x)).isna()==False)&(shp.SIG_KOR_NM.apply(lambda x: re.search(sgg,x)).isna()==False))].centroid.tolist()[0] for mega,sgg in zip(megas,sggs)]

df['centroid'] = centroid

In [266]:
df.head(1)

,Name,시도명,시군구명,centroid,geometry,row_num,x,y
1,XXX,서울시,양천구,POINT (943049.1483472434 1947471.644039042),POINT (943049.148 1947471.644),1,943049.148347,1.947472e+06


In [184]:
df.to_csv("당신이 사는곳을.csv")

In [ ]:
# GeoPandas 변환

In [196]:
df.centroid = df.centroid.apply(lambda x: loads(str(x)))

In [203]:
gdf = gpd.GeoDataFrame(df,geometry = centroid,crs = 5179)

In [ ]:
# 겹치는 지역에 대한 처리 

In [229]:
gdf['row_num'] = gdf.groupby(['시도명','시군구명']).cumcount() + 1

In [231]:
gdf.geometry.x + 50

1     943099.148347
2     940241.387102
3     977441.327671
4     960745.935757
5     950520.018700
6     955534.195298
7     931209.023816
8     951001.786617
9     947930.109711
10    961420.403123
11    945241.295145
12    954098.023714
13    948777.213309
14    951579.628126
15    974066.230835
16    950520.018700
17    931975.208986
18    925537.829393
19    940241.387102
dtype: float64

In [235]:
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y

In [241]:
gdf.loc[gdf.row_num == 2,'x'] = gdf.loc[gdf.row_num == 2,'x'] + 50

In [246]:
gdf2 = gpd.GeoDataFrame(gdf,geometry = [Point(x,y) for x,y in zip(gdf.x,gdf.y)],crs = 5179)

In [ ]:
# 좌표변환 

In [247]:
gdf2 = gdf2.to_crs(4326)

In [249]:
gdf2['lon'] = gdf2.geometry.x
gdf2['lat'] = gdf2.geometry.y

In [267]:
fig = px.scatter_mapbox(gdf2,lat = 'lat',lon = 'lon',hover_name = 'Name',mapbox_style = 'open-street-map')

In [264]:
plot(fig, filename='notion_update2.html')

'notion_update2.html'